https://edinburghcyclehire.com/
Souřadnnice sanic jsou v Decimal degrees in WGS84. Pro použití na openstreet.org mapách jsou přepočteny na pseudo Mercator projekci.



### Příprava dat:
#### Identifikace chybných hodnot 
Zdrojová data obsahují nekonzistence v názvech a popisech stanic. Jejich zeměpisná šířka a délka také není u všech údajů stejná (odchylka GPS).\
Odstraněno seskupením stanic dle jejich id a výpočtem průměru souřadnic.


#### Redukce počtu proměnných
##### Blízké klastry
- sloučením stanic vzdálených od sebe méně než 25 metrů se sníží počet stanic o 45 (22,5%). Odstraněno přepsáním v df rides, vymazáním ze df stations, zůstane ta s větším počtem použití.

##### Vzdálené klastry
- odstraněním stanic vzdálených od sebe více než 50 kilometrů se sníží počet stanic o 1 (Liverpool) - optimalizujeme následný výpočet klastrů. Odstraněno vymazáním v df rides a df stations.  

##### Málo četné a dočasné klastry
 - odstraněním stanic použitých méně než 10 x (pro stanice starší 90 dnů) se sníží počet stanic o 8 (4%). Odstraněno vymazáním v df rides a df stations.
 - odstraněním dočasných stanic ("19th to 23rd June", "festival" a "event") se sníží počet stanic o 6 (3%). Odstraněno vymazáním v df rides a df stations.

Celkem se podařilo snížit množství stanic pro následnou analýzu o 60 t.j. 30 %.




lze načíst výšku stanic z openstreetmap.org ?
k-means

První krok modelování - závěry z obvykle nesupervizovaného modelu jsou vstupem pro následné supervizované modelování
Identifikace podezřelých případů 



In [ ]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
from bokeh.io import output_notebook, show
from bokeh.models import WMTSTileSource
from bokeh.plotting import figure, ColumnDataSource
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 500)
pd.set_option("display.precision", 14)
output_notebook()

In [ ]:
# vyloučit 3 "jízdy" s end stanicí 280 (Exhibition Centre Liverpool)
rides = pd.read_csv("201809-202010.csv", parse_dates=[0,1], usecols = ["started_at","ended_at","duration","start_station_id", "end_station_id"])    

In [ ]:
# Vytvoření tabulky obsahující stanice = načtení z csv a dopočítání vzdálenosti mezi stanicemi.
# 
# Vymazána ? koncová stanice id=280 iloc=30 - Exhibition Centre Liverpool (výstava kol).
# Celková tabulka stations je vytvořena spojením informací ze "start" a "end" slouců. Duplicity jsou řešeny vybráním prvního popisu a průměrem souřadnic - vše dle id.
# Další nekonzistencí je existence více stanic na jednom místě. Vyberu ty, které mají více použití.
# Načtení informací o "start" a "end" stanicích:
# Některé stanice jsou dočasné během akcí - zatím njsou dohromady s ostatními
stations = pd.read_csv("201809-202010.csv", index_col = "start_station_id", usecols = ["start_station_id","start_station_name","start_station_description","start_station_latitude", "start_station_longitude"])   
end_stations = pd.read_csv("201809-202010.csv", index_col = "end_station_id", usecols = ["end_station_id","end_station_name","end_station_description","end_station_latitude", "end_station_longitude"])  
# Přejmenování sloupců a indexu:
column_names = {"start_station_name":"name","start_station_description":"description","start_station_latitude":"latitude", "start_station_longitude":"longitude"}
stations.rename(columns=column_names, index={"start_station_id": "idd"}, inplace=True)
stations.index.names = ['id']
end_column_names = {"end_station_name":"name","end_station_description":"description","end_station_latitude":"latitude", "end_station_longitude":"longitude"}
end_stations.rename(columns=end_column_names, index={'end_station_id': 'id'}, inplace=True)
end_stations.index.names = ['id']
# Sloučení informací o "start" a "end" stanicích a jejich setřídění:
stations = stations.append(end_stations)
stations.sort_index(axis="index", inplace = True)
# Výpočet průměrných koorinátů a přidání k prvním názvům a popisům stanic:
stations_coord_mean = stations.groupby(["id"]).mean()
stations = stations.groupby(["id"]).first()
stations["latitude"] = stations_coord_mean["latitude"]
stations["longitude"] = stations_coord_mean["longitude"]
stations["description"].fillna("", inplace=True)
# Vymazána koncová stanice id 280 - Exhibition Centre Liverpool (výstava kol). Lepším řešením je vymazat "stanice", které jsou vzdáleny od Edinburgu víc než 50? km.
# stations.drop(280, inplace=True)
# přepočet wgs84 souřadnic na  Mercator projekci pro použití v WMTS tile map v openstreetmap.org 
stations["mer_x"] = stations["longitude"] * (6378137 * np.pi/180.0)
stations["mer_y"] = np.log(np.tan((90 + stations["latitude"]) * np.pi/360.0)) * 6378137
# Vymazání dočasných df:
end_stations = end_stations[0:0]
stations_coord_mean = stations_coord_mean[0:0]

In [ ]:
# Výpočet první použití stanic:
stations["first_ride"] = rides.groupby(["start_station_id"]).min()[["started_at"]] 
stations["first_ride_tmp"] = rides.groupby(["end_station_id"]).min()[["started_at"]] 
mfilter = (stations["first_ride"] > stations["first_ride_tmp"]) | (stations["first_ride"].isnull())
stations.loc[mfilter, "first_ride"] = stations.loc[mfilter, "first_ride_tmp"]
stations.drop("first_ride_tmp", axis=1, inplace=True)
# Výpočet počtu použití stanic:
stations["number_of_rides"] = rides.groupby(["start_station_id"])[["duration"]].count()
stations["number_of_rides"].fillna(0, inplace = True)
stations["number_of_rides_tmp"] = rides.groupby(["end_station_id"])[["duration"]].count()
stations["number_of_rides_tmp"].fillna(0, inplace = True)
stations["number_of_rides"] = stations["number_of_rides"].astype(int) + stations["number_of_rides_tmp"].astype(int)
stations.drop("number_of_rides_tmp", axis=1, inplace=True)
# Výpočet vzdáleností mezi stanicemi
# dx = 2*pi*R/360*(l2-l1); l = odmocnina(d1*d1+ d2*d2)
for row2 in range(len(stations.index)): 
    stations[stations.index[row2]] = -1        # np.NaN vnáží do vlastností sloupce "float"
    lat2 = stations.iloc[row2, 2]
    lon2 = stations.iloc[row2, 3]
    for row1 in range(len(stations.index)):
        if row1 > row2 :                       # počítám pouze první část vzdáleností - je to rychlejší a lépe se pak vyhledává 
            lat1 = stations.iloc[row1, 2]
            lon1 = stations.iloc[row1, 3]
            distance = round(np.sqrt(np.square((lat2 - lat1) * 2 * np.pi * 6378137 / 360) + np.square((lon2 - lon1) * 2 * np.pi * 6378137 / 360)))
            stations.iloc[row1, row2+8] = distance.astype(int)

In [ ]:
# Vymazání stanic obsahující v name a description slova "(19th to 23rd June)", "festival" a "event"
idx = stations.index[(stations["name"] + stations["description"]).str.lower().str.find("(19th to 23rd June)") > -1]\
.append(stations.index[(stations["name"] + stations["description"]).str.lower().str.find("festival") > -1])\
.append(stations.index[(stations["name"] + stations["description"]).str.lower().str.find("event") > -1])                 # -1 = nenalezeno
stations.drop(index=idx, inplace=True)
rides.drop(index = rides.index[rides["start_station_id"].isin(idx) | rides["end_station_id"].isin(idx)], inplace=True)

# Vymazání stanic, které mají počet použití menší než 10:
idx = stations.index[(stations["number_of_rides"] < 10) & (stations["first_ride"] < (pd.Timestamp.utcnow() - pd.Timedelta(90, unit="D")) )]
stations.drop(index=idx, inplace=True)
rides.drop(index = rides.index[rides["start_station_id"].isin(idx) | rides["end_station_id"].isin(idx)], inplace=True)


In [ ]:
# Stanice, které jsou blíže než 25 metrů od sebe:
s = []
for col in range(8,len(stations.index)):
    for row in range(len(stations.index)):
        if (stations.iloc[row, col] < 25) & (stations.iloc[row, col] > 0): 
            s.append(row)
stations.iloc[s]

In [ ]:
stations.index[38]

In [ ]:
# Vymazání stanic, které jsou déle než 50 kilometrů od sebe:
s = []
for col in range(8,len(stations.index)):
    for row in range(len(stations.index)):
        if (stations.iloc[row, col] > 50000) & (stations.iloc[row, col] > 0): 
            s.append(( row, col, stations.iloc[row, col] ))
s 

In [ ]:
# k-means clustering
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans
from matplotlib import pyplot

TOOLTIPS = [("id", "@{id}"), ("(x,y)", "(@{latitude}, @{longitude})"), ("name", "@{name}"), ("desc", "@{description}"), ("rides", "@{number_of_rides}"),]
p = figure(plot_width=1200, plot_height=700, tools='reset, pan, wheel_zoom, save, box_zoom', x_range=(-380000,-340000), y_range=(7545000,7553000), 
           x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS, title="Edinburg - Just Eat Cycles - 20 klastrů KMeans") # Edinburg
p.add_tile(WMTSTileSource(url="https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"))

# define dataset
X = stations[["mer_x", "mer_y"]].values.tolist()
W = stations["number_of_rides"].values.tolist()
#X, _ = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=4)
# define the model
model = KMeans(n_clusters=20)
# fit the model
model.fit(X, sample_weight=W)
# assign a cluster to each example
yhat = model.predict(X, sample_weight=W)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = where(yhat == cluster)
    row_ix = row_ix[0].tolist()
    # create scatter of these samples
#    pyplot.scatter(X[row_ix[0]][0], X[row_ix[0]][1])
    p.scatter(X[row_ix[0]][0], X[row_ix[0]][1], size=W[row_ix[0]]/100, color="blue", alpha=0.3)    # vybírám pouze část sloupců, protože bokeh nemůže mít názvy sloupců čísla !!!
    p.scatter(X[row_ix[0]][0], X[row_ix[0]][1], size=30, color="yellow", alpha=1)

p.circle("mer_x", "mer_y", size=8, color="red", alpha=1, source=ColumnDataSource(data=stations.iloc[:,0:8]))    
show(p)

#pyplot.show()

In [ ]:
from sklearn.neighbors import KDTree
import numpy as np
#X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
X = stations[["mer_x", "mer_y"]].values
kdt = KDTree(X, leaf_size=30, metric='euclidean')
a,b = kdt.query(X, k=2, return_distance=True)


In [ ]:
#Query for neighbors within a given radius
import numpy as np
X = stations[["mer_x", "mer_y"]].values
tree = KDTree(X, leaf_size=30, metric='euclidean')     # doctest: +SKIP
print(tree.query_radius(X[:1], r=25, count_only=True))

ind = tree.query_radius(X, r=25, return_distance=True)  # doctest: +SKIP
print(ind)  # indices of neighbors within distance 0.3
ind[0][0]

In [ ]:
ind

In [ ]:
# openstreetmap používá mercator coordináty
TOOLTIPS = [("id", "@{id}"), ("(x,y)", "(@{latitude}, @{longitude})"), ("name", "@{name}"), ("desc", "@{description}"), ("rides", "@{number_of_rides}"),]
p = figure(plot_width=1200, plot_height=700, tools='reset, pan, wheel_zoom, save, box_zoom', x_range=(-380000,-340000), y_range=(7545000,7553000), 
           x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS, title="Edinburg - Just Eat Cycles") # Edinburg
p.add_tile(WMTSTileSource(url="https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"))
p.circle("mer_x", "mer_y", size=8, color="red", alpha=1, source=ColumnDataSource(data=stations.iloc[:,0:8]))    # vybírám pouze část sloupců, protože bokeh nemůže mít názvy sloupců čísla !!!
show(p)